In [1]:
!pip install opencv-python pycocotools numpy > /dev/null 2>&1

! mkdir img_folder > /dev/null 2>&1
! mkdir mask_folder > /dev/null 2>&1
! mkdir output_folder > /dev/null 2>&1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Pasos

Poner imagenes en img_folder y mascaras en mask_folder
Luego se guarda todo en output_folder.

In [3]:
import cv2
import numpy as np
import json
import os
import base64

def mask_to_polygons(binary_mask):
    contours, _ = cv2.findContours(
        binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    polygons = []
    for contour in contours:
        if len(contour) >= 3:
            polygon = contour.squeeze().astype(float).tolist()
            polygons.append(polygon)
    return polygons

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        encoded_string = base64.b64encode(img_file.read()).decode('utf-8')
    return encoded_string

def grayscale_mask_to_labelme_json(mask_path, output_json_path, img_path, label_map=None):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    height, width = mask.shape

    if label_map is None:
        unique_vals = np.unique(mask)
        label_map = {int(v): f"class_{int(v)}" for v in unique_vals}

    shapes = []
    for pixel_value, label in label_map.items():
        binary_mask = (mask == pixel_value).astype(np.uint8)
        polygons = mask_to_polygons(binary_mask)
        for points in polygons:
            shapes.append({
                "label": label,
                "points": points,
                "group_id": None,
                "description": "",
                "shape_type": "polygon",
                "flags": {},
                "mask": None
            })

    base64_image = encode_image_to_base64(img_path)

    labelme_json = {
        "version": "5.8.1",
        "flags": {},
        "shapes": shapes,
        "imagePath": os.path.basename(img_path),
        "imageData": base64_image,
        "imageHeight": height,
        "imageWidth": width
    }

    with open(output_json_path, "w") as f:
        json.dump(labelme_json, f, indent=4)



Especifiar de donde salen / a donde van, los resultados e imagenes.

In [4]:
# === Batch processing setup ===

img_folder = "/content/img_folder"
mask_folder = "/content/drive/MyDrive/FACULTAD/TSCF/OurDataset/grayscale_folder"
output_folder = "/content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe"
os.makedirs(output_folder, exist_ok=True)

label_map = {
    0: "road",
    1: "vegetation",
    2: "sky",
    3: "obstacle",
    4: "other"
}

for img_name in os.listdir(img_folder):
    if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    img_path = os.path.join(img_folder, img_name)

    # Assume all masks are PNGs named the same as the image (without extension)
    mask_filename = os.path.splitext(img_name)[0] + ".png"
    mask_path = os.path.join(mask_folder, mask_filename)

    if not os.path.exists(mask_path):
        print(f"⚠️ Mask not found for image: {img_name}")
        continue

    output_json_path = os.path.join(output_folder, os.path.splitext(img_name)[0] + ".json")

    grayscale_mask_to_labelme_json(
        mask_path,
        output_json_path,
        img_path,
        label_map
    )

    print(f"✅ Processed {img_name} -> {output_json_path}")

✅ Processed 220706_0027.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0027.json
✅ Processed 220706_0082.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0082.json
✅ Processed 220706_0044.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0044.json
✅ Processed 220706_0122.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0122.json
✅ Processed 220706_0008.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0008.json
✅ Processed 220706_0071.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0071.json
✅ Processed 220706_0094.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0094.json
✅ Processed 220706_0076.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0076.json
✅ Processed 220706_0021.png -> /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/labelMe/220706_0021.json
✅ Processed 220706_0107.png -> /content/drive/MyDrive/FACULTAD/T